In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='7e5a4f6a-9ecd-4270-8468-bea207a026e7', project_access_token='p-a505111d3aef8a11263e099902c8c5e6f4d3b7dc')
pc = project.project_context


# Watson Assistant Performance Metrics - English

This notebook shows how you can import Pythion packages and code from a GitHub repo into your Jupyter notebook into Watson Studio or IBM Cloud Pak for Data.

### Table of Contents
* [0.0 Housekeeping](#housekeeping)


## ADD PROJECT TOKEN

This Notebook uses Watson Studio project token. In a newly created project or if your existing Watson Studio project does not already have project token setup, make sure to set that up first.

Here are the steps:

Go to the “Settings” tab of your project and scroll to “Access tokens” and create an access token by clicking on “New token”. Provide a name eg – “For-WA-Tooling” give “Editor” access permissions.

(Complete instructions https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/token.html)

Add the project token to this Notebook:

1. Click on the three vertical dots in the top menu bar and then  "Insert project Token". (Read below if you don't have a project token setup yet.)
2. A new project token for your project should appear in the first cell.
3. The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
4. Make sure to "Run' the cell with your token to initialize the project token into variables.

Example of the first cell (project token cell)
 ```@hidden_cell
 from project_lib import Project
 project = Project(project_id='d65cd0bf-xxxx-xxx-xxx-xxxx-xxx-xxxxx-', project_access_token='p-723xxxxxxxxxxyriuhdd8dd3oxxxxxxxxxxd')
 pc = project.project_context```


In [2]:
#
# Project token would be here.....
#

## 0.0 Housekeeping <a class="anchor" id="housekeeping"></a>
This section will import libraries and dependencies for this notebook. 

In [3]:
from datetime import datetime, timedelta, date
import os

### 0.1 Get Needed Helper Routines

You do not need to change anything in these function definitions, just run the cells to initilize them

In [4]:
def copy_project_asset_file_to_python_env(project=project, asset_filename='asset_file_in_project', python_env_target_dir='python environemnt target directory'):
    filename_with_path=python_env_target_dir+'/'+asset_filename
    #print(os.listdir(python_env_target_dir))
    
    print('Copying file {0} from project assets to python environment'.format(asset_filename))
    file_from_project=project.get_file(asset_filename)
    file_to_write_local = open(filename_with_path, 'wb')
    file_to_write_local.write(file_from_project.read())
    file_to_write_local.close()
    file_from_project.close()

    print('Done.')
    print('Target directory listing: ')
    print(os.listdir(python_env_target_dir))
    print('File: ')
    print(os.stat(filename_with_path))

In [5]:
def copy_python_env_file_to_project_asset(project=project, python_env_source_dir='python environemnt source directory', python_env_filename='python environment file name'):
    filename_with_path=python_env_source_dir+'/'+python_env_filename
    print('Source directory listing: ')
    print(os.listdir(python_env_source_dir))
    print('File: ')
    print(os.stat(filename_with_path))
    print('Copying file {0} from python environment to project assets'.format(python_env_filename))
    file_data=open(filename_with_path, 'rb')
    project.save_data(data=file_data.read(),file_name=python_env_filename,overwrite=True)
    file_data.close() 
    print('Done.')
    #print('Directory listing: ' + os.listdir(python_env_source_dir)

In [6]:
def copy_python_env_data_results_zip_to_project_asset(project=project, data_results_zip_filename='zip file name'):
    
    #create a zip file of the data directory
    print('Verify data directoy was created/exists')
    print('Data directory listing: ')
    print(os.listdir(WA_TOOL_DATA_DIR_W_PATH))
  
    # !echo $WA_TOOL_INSTALL_DIR_W_PATH/$WA_TOOL_RESULTS_TARGZ_FILE $WA_TOOL_DATA_DIR_W_PATH

    print('Creating data zip file...')
    #subprocess.call(["tar", "-cvf", WA_TOOL_RESULTS_TARGZ_FILE, WA_TOOL_DATA_DIR_W_PATH])
    !tar -czvf $WA_TOOL_INSTALL_DIR_W_PATH/$WA_TOOL_RESULTS_TARGZ_FILE $WA_TOOL_DATA_DIR_W_PATH

    print('Verify .tar.gz file was created...')
    #check for the file in the list
    print('Directory listing. Check for the zip file: ')
    print(os.listdir(WA_TOOL_INSTALL_DIR_W_PATH))
    print('Zip file: ')
    print(os.stat(WA_TOOL_INSTALL_DIR_W_PATH+'/'+WA_TOOL_RESULTS_TARGZ_FILE))

    #copy thezip file to project asset
    print('Copying data zip file to project asset.')
    copy_python_env_file_to_project_asset(project, WA_TOOL_INSTALL_DIR_W_PATH, WA_TOOL_RESULTS_TARGZ_FILE)

In [7]:
def copy_test_input_files_from_assets_to_python_env():
    
    #Depending on the test type copy  input files from the project asset to the local python env
    
    #If kfold no files needed
    #If blind or test we need test_input_file
    
    if ( PARAMS['mode'].lower() == 'kfold'):
        #do nothing
        print("No input files needed to copy for kfold.")
    if ( PARAMS['mode'].lower() == 'blind'):   
        # copy PARAMS['test_input_file'] to python env        
        print("Copying input file for blind test to python env...")
        copy_project_asset_file_to_python_env(project, PARAMS['test_input_file'], WA_TOOL_DATA_DIR_W_PATH)       
    if ( PARAMS['mode'].lower() == 'test'):   
        # copy PARAMS['test_input_file'] to python env
        print("Copying input file for standard test to python env...")
        copy_project_asset_file_to_python_env(project, PARAMS['test_input_file'], WA_TOOL_DATA_DIR_W_PATH)       

    ##Not using the feature of previous tests. 
    #Add this to PARAMS For later enhancement
    #'previous_blind_out' : 'NA', #; Previous blind test output file or NA. 'prev-test-out.csv | 'NA'
    #if ( PARAMS['previous_blind_out'].lower() != 'na'):   
        #PARAMS['previous_blind_out']
        #print("Copying previous_blind_out input file to python env...")
        #copy_project_asset_file_to_python_env(project, PARAMS['previous_blind_out'], WA_TOOL_DATA_DIR_W_PATH)       

    if ( PARAMS['partial_credit_table'].lower() != 'na'):   
        #PARAMS['partial_credit_table']
        print("Copying partial_credit_table input file to python env...")
        copy_project_asset_file_to_python_env(project, PARAMS['partial_credit_table'], WA_TOOL_DATA_DIR_W_PATH)


### 0.2 Get Needed Python files

This will do a git clone of the WA_Tool, and then upload `getAllLogs.py` and `extractConversations.py` into your project's assets. The WA_Tool Git project can be found at https://github.com/cognitive-catalyst/WA-Testing-Tool

In [8]:
# Initialize advanced configuration variables
# You do not need to change anything in these unless for advanced usage custom configuration
# just run the cell to initilize them
WS_USER_WORK_DIR_W_PATH= os.getcwd() #'/home/dsxuser/work' #Default Watson Studio user work directory
WA_TOOL_NOTEBOOK_DIR_W_PATH=WS_USER_WORK_DIR_W_PATH+'/'+'wa-tooling-notebook' 
WA_TOOL_INSTALL_DIR_W_PATH=WA_TOOL_NOTEBOOK_DIR_W_PATH+'/'+'WA-Testing-Tool' #directory where git clone extracts the tooling, default name; 
WA_TOOL_DATA_DIR_W_PATH=WA_TOOL_INSTALL_DIR_W_PATH+'/'+'data'
WA_TOOL_RESULTS_TARGZ_FILE='results_data.tar.gz'
WA_TOOL_GITHUB_URL='https://github.com/cognitive-catalyst/WA-Testing-Tool.git'
WA_TOOL_CONFIG_FILE_W_PATH = WA_TOOL_INSTALL_DIR_W_PATH +'/'+'config.ini'
#
WA_TOOL_ANALYTICS_DIR=WA_TOOL_INSTALL_DIR_W_PATH+'/'+'log_analytics'
#
WA_TOOL_GETALLLOGS_FILE_W_PATH=WA_TOOL_ANALYTICS_DIR+'/'+'getAllLogs.py'
WA_TOOL_EXTRACTCONV_FILE_W_PATH=WA_TOOL_ANALYTICS_DIR+'/'+'extractConversations.py'
#
WA_TOOL_GETALLLOGS_FILE_TARGET=WS_USER_WORK_DIR_W_PATH+'/'+'getAllLogs.py'
WA_TOOL_EXTRACTCONV_FILE_TARGET=WS_USER_WORK_DIR_W_PATH+'/'+'extractConversations.py'

#To get current home directory 
print('\nCurrent home directory:')
print(os.getcwd())

#To get current working directory and install directory contents
print('\nCurrent working directory contents:')
!ls -ltr $WS_USER_WORK_DIR_W_PATH

# Make working dir for WA tooling as a sub dir of this WS work directory 
# Ignore if this already exists. It was created the first time
if not (os.path.exists(WA_TOOL_NOTEBOOK_DIR_W_PATH)): os.mkdir(WA_TOOL_NOTEBOOK_DIR_W_PATH)
    
# Git clone the WA-Tooling to an install directory
# Ignore if this already exists. It was cloned if this is not the first time
!git clone $WA_TOOL_GITHUB_URL $WA_TOOL_INSTALL_DIR_W_PATH


Current home directory:
/home/wsuser/work

Current working directory contents:
total 0
Cloning into '/home/wsuser/work/wa-tooling-notebook/WA-Testing-Tool'...
remote: Enumerating objects: 1655, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1655 (delta 34), reused 44 (delta 25), pack-reused 1592
Receiving objects: 100% (1655/1655), 3.45 MiB | 18.69 MiB/s, done.
Resolving deltas: 100% (1023/1023), done.


In [9]:
# Import dependencies. Ensure these are loaded into your Studio assets.
#fobj = open("getAllLogs.py", "wb")
#fobj.write(project.get_file("getAllLogs.py").read()) 
#fobj.close()

!cp -f -v $WA_TOOL_GETALLLOGS_FILE_W_PATH $WA_TOOL_GETALLLOGS_FILE_TARGET

#fobj = open("extractConversations.py", "wb")
#fobj.write(project.get_file("extractConversations.py").read()) 
#fobj.close()

!cp -f -v $WA_TOOL_EXTRACTCONV_FILE_W_PATH $WA_TOOL_EXTRACTCONV_FILE_TARGET

'/home/wsuser/work/wa-tooling-notebook/WA-Testing-Tool/log_analytics/getAllLogs.py' -> '/home/wsuser/work/getAllLogs.py'
'/home/wsuser/work/wa-tooling-notebook/WA-Testing-Tool/log_analytics/extractConversations.py' -> '/home/wsuser/work/extractConversations.py'


### 0.3 Import Needed Python libs
This step takes a minmute or two - make sure that you see the **PYTHON ENVIRONMENT IMPORTS COMPLETED** message before continuing on.

In [10]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter("ignore")

!pip install ibm-watson
!pip install --user --upgrade "plotly_express==0.4.0";
!pip install --user --upgrade "matplotlib==3.2.1";
!pip install squarify
!pip install plotly

import json
import pandas as pd
import getAllLogs
import extractConversations
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt 

print('\n PYTHON ENVIRONMENT IMPORTS COMPLETED \n')


 PYTHON ENVIRONMENT IMPORTS COMPLETED 

